## 2.7 Extension de tipos de datos 

Pandas se construyó originalmente sobre las capacidades presentes en NumPy, una librería de cálculo de arrays utilizada principalmente para trabajar con datos numéricos. Muchos conceptos de pandas, como los datos que faltan, se implementaron utilizando lo que estaba disponible en NumPy mientras se intentaba maximizar la compatibilidad entre las bibliotecas que utilizaban conjuntamente NumPy y Pandas.

Basarse en NumPy conllevó una serie de deficiencias, como:

El manejo de datos faltantes para algunos tipos de datos numéricos, como enteros y booleanos, era incompleto. Como resultado, cuando se introducían datos perdidos en tales datos, pandas convertía el tipo de datos a float64 y utilizaba np.nan para representar valores nulos. Esto tenía efectos agravantes al introducir problemas sutiles en muchos algoritmos de pandas.

Los conjuntos de datos con muchos datos de cadenas eran costosos computacionalmente y utilizaban mucha memoria.

Algunos tipos de datos, como intervalos de tiempo, timedeltas, y timestamps con zonas horarias, no podían ser soportados eficientemente sin usar arrays de objetos Python computacionalmente caros.

Más recientemente, pandas ha desarrollado un sistema de tipos de extensión que permite añadir nuevos tipos de datos aunque no estén soportados nativamente por NumPy. Estos nuevos tipos de datos pueden ser tratados como de primera clase junto con los datos procedentes de arrays NumPy.

Veamos un ejemplo en el que creamos una Serie de enteros con un valor perdido:

In [115]:
import pandas as pd
import numpy as np

In [116]:
s = pd.Series([1, 2, 3, None])
s

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

In [117]:
s_2 = pd.Series([1,2,3])
s_2

0    1
1    2
2    3
dtype: int64

Por defecto, pandas tratará de inferir el tipo de datos de la serie.
Dado que hay un `None` en la lista, pandas convertirá automáticamente el tipo de datos a float64 porque el tipo float en pandas puede manejar valores nulos (NaN), mientras que el tipo `int` tradicional no puede.

In [118]:
s.dtype

dtype('float64')

Principalmente por razones de retrocompatibilidad, Series utiliza el comportamiento heredado de usar un tipo de datos float64 y np.nan para el valor perdido. Podríamos crear esta Serie en su lugar usando pandas.Int64Dtype:

In [119]:
s_1 = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())
s_1

0       1
1       2
2       3
3    <NA>
dtype: Int64

Aquí, especificamos explícitamente que queremos usar el tipo de datos Int64 de pandas, que es un tipo de datos de enteros que soporta valores nulos.
Esto permite que la serie mantenga su tipo de datos como enteros (Int64), incluso con la presencia de valores nulos.
Los valores nulos se representan con <NA>, que es un valor nulo especial compatible con el tipo Int64.

In [120]:
s_1.dtype

Int64Dtype()

La salida `<NA>` indica que falta un valor para un array de tipo extensión. Esto utiliza el valor centinela especial `pandas.NA`:

In [121]:
s_1[3]

<NA>

In [122]:
s_1[3] is pd.NA

True

In [123]:
s[3] is pd.NA

False

También podríamos haber utilizado la abreviatura `"Int64"` en lugar de `pd.Int64Dtype()` para especificar el tipo. La mayúscula es necesaria, de lo contrario será un tipo sin extensión basado en NumPy:

In [124]:
s_3 = pd.Series([1, 2, 3, None], dtype="Int64")
s_3

0       1
1       2
2       3
3    <NA>
dtype: Int64

Pandas también tiene un tipo de extensión especializado para datos de cadena que no utiliza arrays de objetos NumPy (requiere la biblioteca pyarrow, que puede que necesite instalar por separado):

In [125]:
se = pd.Series(['one', 'two', None, 'three'])
se

0      one
1      two
2     None
3    three
dtype: object

In [126]:
s_4 = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s_4

0      one
1      two
2     <NA>
3    three
dtype: string

Estos arrays de cadenas suelen utilizar mucha menos memoria y, con frecuencia, son más eficientes desde el punto de vista computacional para realizar operaciones en grandes conjuntos de datos.

En la siguiente Tabla figura una lista de algunos de los tipos de extensión disponibles. 

**Pandas extension data types**

`BooleanDtype`: Datos booleanos anulables, utilice `"boolean"` al pasarlos como cadena.

`CategoricalDtype`: Tipo de dato categórico, utilice `"category"` cuando pase como cadena

`DatetimeTZDtype`: Datetime with time zone

`Float32Dtype`: Coma flotante anulable 32-bit , use "Float32" cuando pase como cadena.

`Float64Dtype`: Coma flotante anulable de 64 bits, utilice "Float64" al pasar como cadena.

`Int8Dtype`: Entero con signo anulable de 8 bits, utilice "Int8" al pasarlo como cadena.

`Int16Dtype`: Entero con signo anulable de 16 bits, utilice "Int16" al pasarlo como cadena.

`Int32Dtype`: Entero con signo anulable de 32 bits, utilice "Int32" al pasarlo como cadena.

`Int64Dtype`: Entero con signo anulable de 64 bits, utilice "Int64" al pasarlo como cadena.

`UInt8Dtype`: Entero sin signo anulable de 8 bits, utilice "UInt8" al pasarlo como cadena.

`UInt16Dtype`: Entero sin signo anulable de 16 bits, utilice "UInt16" al pasarlo como cadena.

`UInt32Dtype`: Entero sin signo de 32 bits anulable, utilice "UInt32" al pasarlo como cadena.


`UInt64Dtype`: Entero sin signo de 64 bits anulable, use "UInt64" cuando pase como cadena.

Los tipos de extensión pueden pasarse al método Series `astype`, lo que permite convertirlos fácilmente como parte del proceso de limpieza de datos:

In [127]:
df = pd.DataFrame({"A": [1, 2, None, 4],
                   "B": ["one", "two", "three", None],
                   "C": [False, None, False, True]})

df

,A,B,C
0,1.0,one,False
1,2.0,two,None
2,NaN,three,False
3,4.0,None,True


In [128]:
df["A"] = df["A"].astype("Int64")

In [129]:
df["B"] = df["B"].astype("string")

In [130]:
 df["C"] = df["C"].astype("boolean")

In [131]:
df

,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,<NA>,True


## 2.8 Manipulación de cadenas (string)

Python ha sido durante mucho tiempo un lenguaje popular de manipulación de datos en bruto, en parte debido a su facilidad de uso para el procesamiento de cadenas y texto. La mayoría de las operaciones de texto se simplifican con los métodos incorporados en el objeto string. Para la comparación de patrones y manipulaciones de texto más complejas, pueden ser necesarias expresiones regulares. Pandas se suma a la mezcla al permitirle aplicar expresiones de cadena y regulares de forma concisa en arrays enteros de datos, manejando además la molestia de los datos que faltan.

### Métodos de objetos de cadena incorporados en Python

En muchas aplicaciones de manipulación de cadenas y scripts, los métodos de cadena incorporados son suficientes. Por ejemplo, una cadena separada por comas puede dividirse en trozos con split:

In [132]:
val = "a,b,  guido"
val.split(",")

['a', 'b', '  guido']

`split` se combina a menudo con `strip` para recortar los espacios en blanco (incluidos los saltos de línea):

In [133]:
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'guido']

Estas subcadenas (substrings) podrían concatenarse con un delimitador de dos puntos utilizando la suma:

In [134]:
first, second, third = pieces
first + "::" + second + "::" + third

'a::b::guido'

Pero éste no es un método genérico práctico. Una forma más rápida y pitónica es pasar una lista o tupla al método `join` sobre la cadena "::":

In [135]:
"::".join(pieces)

'a::b::guido'

Otros métodos se ocupan de localizar subcadenas. Utilizar la palabra clave `in` de Python es la mejor forma de detectar una subcadena, aunque también se pueden utilizar `index` y `find`:

In [136]:
"guido" in val

True

In [137]:
val.index(",")

1

In [138]:
val.find(":")

-1

Tenga en cuenta que la diferencia entre `find` e `index` es que index lanza una excepción si no se encuentra la cadena (en lugar de devolver -1):

In [139]:
val.index(":")

ValueError: substring not found

Por su parte, `count` devuelve el número de apariciones de una determinada subcadena:

In [ ]:
val.count(",")

2

`replace` sustituirá las ocurrencias de un patrón por otro. También se suele utilizar para eliminar patrones, pasando una cadena vacía:

In [ ]:
val.replace(",", "::")

'a::b::  guido'

In [ ]:
val.replace(",", "")

'ab  guido'

En el siguiente enlace puede encontrar mas [metodos de cadena]('https://www.w3schools.com/python/python_ref_string.asp')

### Expresiones Regulares

Las expresiones regulares proporcionan una forma flexible de buscar o hacer coincidir patrones de cadenas (a menudo más complejos) en el texto. Una expresión regular, comúnmente llamada regex, es una cadena formada según el lenguaje de expresiones regulares. El módulo `re` incorporado en Python se encarga de aplicar expresiones regulares a las cadenas; aquí daré varios ejemplos de su uso.

Las funciones del módulo `re` se dividen en tres categorías: coincidencia de patrones, sustitución y división. Naturalmente, todas ellas están relacionadas; un `regex` describe un patrón a localizar en el texto, que luego puede utilizarse para muchos fines. Veamos un ejemplo sencillo: supongamos que queremos dividir una cadena con un número variable de caracteres de espacio en blanco (tabuladores, espacios y nuevas líneas).

La expresión regular que describe uno o más caracteres de espacio en blanco es `\s+`:

In [ ]:
import re
text = "foo    bar\t baz  \tqux"

re.split(r"\s+", text)

['foo', 'bar', 'baz', 'qux']

Cuando se llama a `re.split(r"\s+", text)`, primero se compila la expresión regular y luego se llama a su método split sobre el texto pasado. Puede compilar la expresión regular usted mismo con `re.compile`, formando un objeto `regex` reutilizable:

In [ ]:
regex = re.compile(r"\s+")

regex.split(text)

['foo', 'bar', 'baz', 'qux']

Si, en cambio, desea obtener una lista de todos los patrones que coincidan con la expresión regular, puede utilizar el método `findall`:

In [ ]:
regex.findall(text)

['    ', '\t ', '  \t']

Para evitar escapes no deseados con \ en una expresión regular, utilice literales de cadena sin procesar como `r"C:\x"` en lugar del equivalente `"C:\\x"`.

La creación de un objeto regex con `re.compile` es muy recomendable si se pretende aplicar la misma expresión a muchas cadenas; de este modo se ahorrarán ciclos de CPU.

`match` y `search` están estrechamente relacionados con `findall`. Mientras que `findall` devuelve todas las coincidencias de una cadena, `search` sólo devuelve la primera coincidencia. Más rígidamente, `match` sólo encuentra coincidencias al principio de la cadena. Como ejemplo menos trivial, consideremos un bloque de texto y una expresión regular capaz de identificar la mayoría de las direcciones de correo electrónico:

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com"""

pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

- `[A-Z0-9._%+-]+`:  
    - `[A-Z0-9._%+-]` es un conjunto de caracteres que incluye letras mayúsculas (A-Z), dígitos (0-9), y los caracteres ., _, %, +, y -.
    - `+` indica que uno o más de estos caracteres pueden aparecer.    
    - `@`: Un carácter de arroba literal.

- `[A-Z0-9.-]+`:
    - `[A-Z0-9.-]` es un conjunto de caracteres que incluye letras mayúsculas (A-Z), dígitos (0-9), y los caracteres . y -.
    - `+` indica que uno o más de estos caracteres pueden aparecer.

- `\.`: Un punto literal (el \ se usa para escapar el . ya que, en regex, el punto normalmente significa "cualquier carácter").

- `[A-Z]{2,4}`:
    - `[A-Z]` es un conjunto de letras mayúsculas.  
    - `{2,4}` indica que deben aparecer de 2 a 4 de estas letras (por ejemplo, .com, .net, .info).

In [ ]:
regex = re.compile(pattern, flags=re.IGNORECASE)
# re.IGNORECASE hace que la expresión regular no distinga entre mayúsculas y minúsculas. 

Esta línea compila el patrón de expresión regular en un objeto regex utilizando la función `re.compile()`. `flags=re.IGNORECASE`  hace que la búsqueda sea insensible a mayúsculas y minúsculas. Es decir, A-Z en el patrón también coincidirá con a-z.

**Utilizando `findall` en el texto se obtiene una lista de las direcciones de correo electrónico**:

In [ ]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

**`search` devuelve un objeto coincidente especial para la primera dirección de correo electrónico del texto. Para la regex anterior, el objeto coincidente sólo puede indicarnos la posición inicial y final del patrón en la cadena:**

In [ ]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

`span=(5, 20)`: span es un tupla que indica los índices de inicio y fin de la coincidencia en la cadena original text.
En este caso, la subcadena que coincide comienza en el índice 5 y termina en el índice 20.

In [ ]:
text[m.start():m.end()]

'dave@google.com'

`regex.match` devuelve `None`, ya que sólo coincidirá si el patrón aparece al principio de la cadena:

In [ ]:
print(regex.match(text))

None


De forma similar, `sub` devolverá una nueva cadena con las ocurrencias del patrón sustituidas por una nueva cadena:

In [ ]:
print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED


Supongamos que desea encontrar direcciones de correo electrónico y, al mismo tiempo, segmentar cada dirección en sus tres componentes: nombre de usuario, nombre de dominio y sufijo de dominio. Para ello, ponga entre paréntesis las partes del patrón que desea segmentar:

In [ ]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"

regex = re.compile(pattern, flags=re.IGNORECASE)

Un objeto match producido por esta regex modificada devuelve una tupla de los componentes del patrón con su método groups:

In [ ]:
m = regex.match("wesm@bright.net")

m.groups()

('wesm', 'bright', 'net')

`findall` devuelve una lista de tuplas cuando el patrón tiene grupos:

In [ ]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

`sub` también tiene acceso a los grupos de cada coincidencia mediante símbolos especiales como `\1` y `\2`. El símbolo `\1` corresponde al primer grupo coincidente, `\2` corresponde al segundo, y así sucesivamente:

In [ ]:
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com


A continuación algunos metodos adicionales de expresiones regulares.

`findall`: Devuelve todos los patrones coincidentes no solapados en una cadena como una lista.

`finditer`: Como findall, pero devuelve un iterador.

`match`: 	Coincide con el patrón al inicio de la cadena y opcionalmente segmenta los componentes del patrón en grupos; si el patrón coincide, devuelve un objeto coincidente, y en caso contrario `None`.

`search`: Escanea la cadena en busca de coincidencias con el patrón, devolviendo un objeto coincidente si es así; a diferencia de match, la coincidencia puede estar en cualquier parte de la cadena en lugar de sólo al principio.

`split`: Rompe la cadena en trozos en cada aparición del patrón.

`sub, subn`: Reemplazar todas (`sub`) o las primeras `n` ocurrencias (`subn`) del patrón en la cadena con la expresión de reemplazo; utilizar los símbolos `\1`, `\2`, ... para referirse a los elementos del grupo de coincidencia en la cadena de reemplazo.

### Funciones de cadena en pandas

Limpiar un conjunto de datos desordenado para su análisis suele requerir mucha manipulación de cadenas. Para complicar las cosas, una columna que contiene cadenas a veces tiene datos que faltan:

In [ ]:
data = {"Dave": "dave@google.com",
        "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com",
        "Wes": np.nan}

data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [ ]:
data.isna()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

Se pueden aplicar métodos de cadenas y expresiones regulares (pasando una `lambda` u otra función) a cada valor utilizando `data.map`, pero fallará en los valores NA (nulos). Para hacer frente a esto, Series dispone de métodos orientados a arrays para operaciones con cadenas que omiten y propagan los valores NA. Se accede a ellos a través del atributo str de Series; por ejemplo, podríamos comprobar si cada dirección de correo electrónico contiene "gmail" con `str.contains`:

In [ ]:
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Tenga en cuenta que el resultado de esta operación tiene un dtype objeto. pandas tiene tipos de extensión que proporcionan un tratamiento especializado de cadenas, enteros y datos booleanos:

In [ ]:
data_as_string_ext = data.astype('string')
data_as_string_ext

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string

In [ ]:
data_as_string_ext.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

También se pueden utilizar expresiones regulares, junto con otras opciones como IGNORECASE:

In [ ]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

Puede utilizar str.get o indexar en el atributo str:

In [ ]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
# busca todas las coincidencias del patrón en cada elemento 
# de la Serie y devuelve listas de tuplas con las coincidencias encontradas.
# .str[0] selecciona la primera coincidencia en cada lista de coincidencias.
matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [ ]:
# matches = data.str.findall(pattern, flags=re.IGNORECASE)
# matches

In [ ]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

De forma similar, puede cortar cadenas utilizando esta sintaxis:

In [ ]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

El método `str.extract` devolverá los grupos capturados de una expresión regular como un DataFrame:

In [ ]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


En la siguiente tabla se muestran otros métodos de String en Series

`cat`: Concatenar cadenas por elementos con un delimitador opcional

`contains`: Devuelve una matriz booleana si cada cadena contiene un patrón/regex.

`count`: Contar ocurrencias del patrón

`extract`: Utilice una expresión regular con grupos para extraer una o varias cadenas de una serie de cadenas; el resultado será un DataFrame con una columna por grupo.

`endswith`: Equivale a x.endswith(pattern) para cada elemento

`startswith`: Equivale a x.startswith(pattern) para cada elemento

 `findall`: Calcula la lista de todas las apariciones del patrón/regex para cada cadena.
 
 `get`: Índice en cada elemento (recuperar el i-ésimo elemento)
 
 `isalnum`: Equivalente a str.alnum incorporado
 
 `isalpha`: Equivalente al built-in str.isalpha.
 
 `isdecimal`: Equivalente al built-in str.isdecimal.

`isdigit`: Equivalente al built-in str.isdigit

`islower`: Equivalent to built-in str.islower

`isnumeric`: Equivalent to built-in str.isnumeric.

`isupper`: Equivalent to built-in str.isupper.

`join`: Une las cadenas de cada elemento de la Serie con el separador pasado.

`len`: Calcular la longitud de cada cadena.

`lower, upper` : Convertir casos; equivalente a x.lower() o x.upper() para cada elemento.

`match`: Utiliza re.match con la expresión regular pasada en cada elemento, devolviendo True o False si coincide

`pad`: Añadir espacios en blanco a la izquierda, a la derecha o a ambos lados de las cadenas

`center`: Equivale a pad(side="both")

`replace`: Reemplazar las ocurrencias de un patrón/regex por otra cadena

`slice`: Corta cada cadena de la serie

`split` : Dividir cadenas según delimitador o expresión regular

`strip`: Recorta los espacios en blanco de ambos lados, incluidas las nuevas líneas.

`rstrip`: Recortar los espacios en blanco de la derecha.

`lstrip`: Recortar los espacios en blanco de la izquierda.

### Ejercicio 5. Expresiones regulares.

#### Dado el siguiente Diccionario:  

In [ ]:

data = {
    'Name': [
        'Dave', 'Steve', 'Rob', 'Ryan', 'Alice', 'Eve', 'John', 'Jane', 'Peter', 'Mary', 'Tom', 'Lucy', 
        'Mike', 'Chris', 'Emma', 'Olivia', 'Sophia', 'Liam', 'Noah', 'Mason', 'Ava', 'Mia', 'James', 'Benjamin'
    ],
    'Email': [
        'dave@google.com', 'steve@gmail.com', 'rob@yahoo.com', 'ryan@gmail.com', 
        'alice@hotmail.com', 'eve@google.com', 'john@outlook.com', 'jane@gmail.com',
        'peter@amazon.com', 'mary@google.com', 'tom@apple.com', 'lucy@yahoo.com',
        'mike@facebook.com', 'chris@netflix.com', 'emma@google.com', 'olivia@gmail.com',
        'sophia@amazon.com', 'liam@apple.com', 'noah@google.com', 'mason@yahoo.com',
        'ava@outlook.com', 'mia@gmail.com', 'james@hotmail.com', 'benjamin@google.com'
    ]
}


### Una vez pasado el diccionario a un DataFrame que contiene nombres y direcciones de correo electrónico, realiza las siguientes tareas:

- Extrae los dominios de los correos electrónicos.
- Cuenta la frecuencia de cada dominio.
- Crea un nuevo DataFrame que contenga cada dominio como columna y el número de veces que se repiten dichos dominios en las filas.
#### Resolver de las dos formas: Usando expresiones regulares y usando los métodos propios de pandas.

## Solución con expresiones regulares

In [ ]:
import re

# Función para extraer el dominio usando expresiones regulares
def extract_domain(email):
    return re.search("@([\w\.]+)", email).group(1)

# Aplicamos la función a la columna 'Email'
df['Domain'] = df['Email'].apply(extract_domain)
print(df)


        Name                Email        Domain
0       Dave      dave@google.com    google.com
1      Steve      steve@gmail.com     gmail.com
2        Rob        rob@yahoo.com     yahoo.com
3       Ryan       ryan@gmail.com     gmail.com
4      Alice    alice@hotmail.com   hotmail.com
5        Eve       eve@google.com    google.com
6       John     john@outlook.com   outlook.com
7       Jane       jane@gmail.com     gmail.com
8      Peter     peter@amazon.com    amazon.com
9       Mary      mary@google.com    google.com
10       Tom        tom@apple.com     apple.com
11      Lucy       lucy@yahoo.com     yahoo.com
12      Mike    mike@facebook.com  facebook.com
13     Chris    chris@netflix.com   netflix.com
14      Emma      emma@google.com    google.com
15    Olivia     olivia@gmail.com     gmail.com
16    Sophia    sophia@amazon.com    amazon.com
17      Liam       liam@apple.com     apple.com
18      Noah      noah@google.com    google.com
19     Mason      mason@yahoo.com     ya

## Solución con los métodos propios de Pandas

In [ ]:
# Extraer el dominio usando el método split de pandas
df['Domain'] = df['Email'].str.split('@').str[1]
print(df)


        Name                Email        Domain
0       Dave      dave@google.com    google.com
1      Steve      steve@gmail.com     gmail.com
2        Rob        rob@yahoo.com     yahoo.com
3       Ryan       ryan@gmail.com     gmail.com
4      Alice    alice@hotmail.com   hotmail.com
5        Eve       eve@google.com    google.com
6       John     john@outlook.com   outlook.com
7       Jane       jane@gmail.com     gmail.com
8      Peter     peter@amazon.com    amazon.com
9       Mary      mary@google.com    google.com
10       Tom        tom@apple.com     apple.com
11      Lucy       lucy@yahoo.com     yahoo.com
12      Mike    mike@facebook.com  facebook.com
13     Chris    chris@netflix.com   netflix.com
14      Emma      emma@google.com    google.com
15    Olivia     olivia@gmail.com     gmail.com
16    Sophia    sophia@amazon.com    amazon.com
17      Liam       liam@apple.com     apple.com
18      Noah      noah@google.com    google.com
19     Mason      mason@yahoo.com     ya

## Contar la frecuencia de cada dominio

In [ ]:
# Contar la frecuencia de cada dominio
domain_counts = df['Domain'].value_counts()
print(domain_counts)


Domain
google.com      6
gmail.com       5
yahoo.com       3
hotmail.com     2
outlook.com     2
amazon.com      2
apple.com       2
facebook.com    1
netflix.com     1
Name: count, dtype: int64


##  Nuevo DataFrame con la frecuencia de cada dominio

## Usando expresiones regulares


In [ ]:
# Crear un nuevo DataFrame con la frecuencia de cada dominio
domain_df = pd.DataFrame(domain_counts).reset_index()
domain_df.columns = ['Domain', 'Count']
print(domain_df)


         Domain  Count
0    google.com      6
1     gmail.com      5
2     yahoo.com      3
3   hotmail.com      2
4   outlook.com      2
5    amazon.com      2
6     apple.com      2
7  facebook.com      1
8   netflix.com      1


## Usando métodos propios de pandas

In [ ]:
# Crear el DataFrame con los dominios y sus cuentas
domain_df = df['Domain'].value_counts().reset_index()
domain_df.columns = ['Domain', 'Count']
print(domain_df)


## Código completo


In [ ]:
import pandas as pd
import re

data = {
    'Name': [
        'Dave', 'Steve', 'Rob', 'Ryan', 'Alice', 'Eve', 'John', 'Jane', 'Peter', 'Mary', 'Tom', 'Lucy', 
        'Mike', 'Chris', 'Emma', 'Olivia', 'Sophia', 'Liam', 'Noah', 'Mason', 'Ava', 'Mia', 'James', 'Benjamin'
    ],
    'Email': [
        'dave@google.com', 'steve@gmail.com', 'rob@yahoo.com', 'ryan@gmail.com', 
        'alice@hotmail.com', 'eve@google.com', 'john@outlook.com', 'jane@gmail.com',
        'peter@amazon.com', 'mary@google.com', 'tom@apple.com', 'lucy@yahoo.com',
        'mike@facebook.com', 'chris@netflix.com', 'emma@google.com', 'olivia@gmail.com',
        'sophia@amazon.com', 'liam@apple.com', 'noah@google.com', 'mason@yahoo.com',
        'ava@outlook.com', 'mia@gmail.com', 'james@hotmail.com', 'benjamin@google.com'
    ]
}

df = pd.DataFrame(data)

# Extraer dominios usando expresiones regulares
def extract_domain(email):
    return re.search("@([\w\.]+)", email).group(1)

df['Domain_regex'] = df['Email'].apply(extract_domain)

# Extraer dominios usando métodos propios de pandas
df['Domain_pandas'] = df['Email'].str.split('@').str[1]

# Contar la frecuencia de cada dominio
domain_counts_regex = df['Domain_regex'].value_counts()
domain_counts_pandas = df['Domain_pandas'].value_counts()

# Crear DataFrame con la frecuencia de cada dominio
domain_df_regex = pd.DataFrame(domain_counts_regex).reset_index()
domain_df_regex.columns = ['Domain', 'Count']

domain_df_pandas = pd.DataFrame(domain_counts_pandas).reset_index()
domain_df_pandas.columns = ['Domain', 'Count']

print("DataFrame con expresiones regulares:")
print(domain_df_regex)
print("\nDataFrame con métodos propios de pandas:")
print(domain_df_pandas)


## 2.9 Datos Categóricos

In [140]:
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values   

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [141]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [142]:
pd.value_counts(values)

C:\Users\CursosTardes\AppData\Local\Temp\ipykernel_12484\3297668723.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(values)


apple     6
orange    2
Name: count, dtype: int64

In [143]:
pd.Series(values).value_counts()

apple     6
orange    2
Name: count, dtype: int64

Muchos sistemas de datos (para almacenamiento de datos, cálculo estadístico u otros usos) han desarrollado enfoques especializados para representar datos con valores repetidos para un almacenamiento y cálculo más eficientes. En el almacenamiento de datos, una práctica recomendada consiste en utilizar las denominadas tablas de dimensiones, que contienen los valores distintos y almacenan las observaciones primarias como claves enteras que hacen referencia a la tabla de dimensiones:

In [144]:
values = pd.Series([0, 1, 0, 0] * 2)
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [145]:
dim = pd.Series(['apple', 'orange'])
dim

0     apple
1    orange
dtype: object

Podemos utilizar el método `take` para restaurar la serie original de cadenas:

In [146]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

Esta representación como números enteros se denomina representación categórica o codificada en diccionario. La array de valores distintos puede denominarse categorías, diccionario o niveles de los datos. Aqui utilizaremos los términos categórico y categorías. Los valores enteros que hacen referencia a las categorías se denominan códigos de categoría o simplemente códigos.

### Ejemplo

Supongamos que tenemos una serie de índices codificados que representan diferentes tipos de mascotas, y una serie de etiquetas que contienen los nombres de las mascotas. Queremos usar el método take para reconstruir las etiquetas originales a partir de los índices.

In [147]:
# Series de índices codificados
encoded_labels = pd.Series([0, 1, 2, 1, 0, 2, 2, 0])

# Series de etiquetas originales
pet_labels = pd.Series(['cat', 'dog', 'bird'])

# Reconstruir las etiquetas originales
decoded_labels = pet_labels.take(encoded_labels)

decoded_labels

0     cat
1     dog
2    bird
1     dog
0     cat
2    bird
2    bird
0     cat
dtype: object

### Ejercicio 6. Dados los siguientes datos:

In [ ]:

encoded_labels = [
    0, 1, 2, 3, 0, 4, 5, 6, 2, 1, 3, 0, 5, 4, 6, 1, 2, 5, 4, 3,
    7, 8, 9, 10, 7, 11, 12, 13, 9, 8, 10, 7, 12, 11, 13, 8, 9, 12, 11, 10,
    14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33
]

animal_labels = [
    'cat', 'dog', 'bird', 'fish', 'lion', 'tiger', 'bear', 'elephant', 
    'giraffe', 'zebra', 'kangaroo', 'koala', 'panda', 'monkey',
    'shark', 'whale', 'dolphin', 'penguin', 'ostrich', 'crocodile', 
    'alligator', 'hippopotamus', 'rhinoceros', 'cheetah', 'leopard', 
    'jaguar', 'wolf', 'fox', 'rabbit', 'squirrel', 'bat', 'owl', 
    'hawk', 'eagle'
]



1- Recrear las etiquetas originales a partir de los índices codificados.  
2- Contar la frecuencia de cada etiqueta de animal.  
3- Identificar las etiquetas de animales únicas presentes en los datos.  
4- Determinar el animal más frecuente y el menos frecuente.  
5- Agrupar los animales y contar la frecuencia por grupos (mamíferos, aves, reptiles, etc.).  
6- Identificar la posición de las primeras y últimas ocurrencias de cada animal.  
7- Generar un resumen estadístico de las frecuencias de los animales (media, mediana, desviación estándar).  
8- Filtrar y mostrar solo los animales con una frecuencia mayor que 2.  
9- Encontrar y mostrar las etiquetas de animales que aparecen solo una vez.  
10- Calcular el porcentaje de apariciones de cada animal respecto al total.  
11- Ordenar las etiquetas de animales por frecuencia en orden descendente.  
12- Generar un DataFrame que muestre la frecuencia acumulativa de las etiquetas.

## 1. Recrear las etiquetas originales a partir de los índices codificados

In [148]:
import pandas as pd
import numpy as np

# Datos proporcionados
encoded_labels = [
    0, 1, 2, 3, 0, 4, 5, 6, 2, 1, 3, 0, 5, 4, 6, 1, 2, 5, 4, 3,
    7, 8, 9, 10, 7, 11, 12, 13, 9, 8, 10, 7, 12, 11, 13, 8, 9, 12, 11, 10,
    14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33
]

animal_labels = [
    'cat', 'dog', 'bird', 'fish', 'lion', 'tiger', 'bear', 'elephant', 
    'giraffe', 'zebra', 'kangaroo', 'koala', 'panda', 'monkey',
    'shark', 'whale', 'dolphin', 'penguin', 'ostrich', 'crocodile', 
    'alligator', 'hippopotamus', 'rhinoceros', 'cheetah', 'leopard', 
    'jaguar', 'wolf', 'fox', 'rabbit', 'squirrel', 'bat', 'owl', 
    'hawk', 'eagle'
]

# Recrear las etiquetas originales
original_labels = [animal_labels[i] for i in encoded_labels]
print(original_labels)


['cat', 'dog', 'bird', 'fish', 'cat', 'lion', 'tiger', 'bear', 'bird', 'dog', 'fish', 'cat', 'tiger', 'lion', 'bear', 'dog', 'bird', 'tiger', 'lion', 'fish', 'elephant', 'giraffe', 'zebra', 'kangaroo', 'elephant', 'koala', 'panda', 'monkey', 'zebra', 'giraffe', 'kangaroo', 'elephant', 'panda', 'koala', 'monkey', 'giraffe', 'zebra', 'panda', 'koala', 'kangaroo', 'shark', 'whale', 'dolphin', 'penguin', 'ostrich', 'crocodile', 'alligator', 'hippopotamus', 'rhinoceros', 'cheetah', 'leopard', 'jaguar', 'wolf', 'fox', 'rabbit', 'squirrel', 'bat', 'owl', 'hawk', 'eagle']


## 2. Contar la frecuencia de cada etiqueta de animal

In [151]:
# Crear un DataFrame para facilitar el análisis
df = pd.DataFrame(original_labels, columns=['Animal'])

# Contar la frecuencia de cada etiqueta de animal
frequency_counts = df['Animal'].value_counts()
print(frequency_counts)


Animal
cat             3
elephant        3
dog             3
panda           3
kangaroo        3
zebra           3
giraffe         3
koala           3
tiger           3
lion            3
fish            3
bird            3
bear            2
monkey          2
leopard         1
hawk            1
owl             1
bat             1
squirrel        1
rabbit          1
fox             1
wolf            1
jaguar          1
penguin         1
cheetah         1
rhinoceros      1
hippopotamus    1
alligator       1
crocodile       1
ostrich         1
dolphin         1
whale           1
shark           1
eagle           1
Name: count, dtype: int64


## 3. Identificar las etiquetas de animales únicas presentes en los datos

In [152]:
# Etiquetas de animales únicas
unique_animals = df['Animal'].unique()
print(unique_animals)


['cat' 'dog' 'bird' 'fish' 'lion' 'tiger' 'bear' 'elephant' 'giraffe'
 'zebra' 'kangaroo' 'koala' 'panda' 'monkey' 'shark' 'whale' 'dolphin'
 'penguin' 'ostrich' 'crocodile' 'alligator' 'hippopotamus' 'rhinoceros'
 'cheetah' 'leopard' 'jaguar' 'wolf' 'fox' 'rabbit' 'squirrel' 'bat' 'owl'
 'hawk' 'eagle']


## 4. Determinar el animal más frecuente y el menos frecuente

In [153]:
# Animal más frecuente
most_frequent_animal = frequency_counts.idxmax()
most_frequent_count = frequency_counts.max()
print(f"Most frequent animal: {most_frequent_animal} with {most_frequent_count} occurrences")

# Animal menos frecuente
least_frequent_animal = frequency_counts.idxmin()
least_frequent_count = frequency_counts.min()
print(f"Least frequent animal: {least_frequent_animal} with {least_frequent_count} occurrence")


Most frequent animal: cat with 3 occurrences
Least frequent animal: leopard with 1 occurrence


## 5. Agrupar los animales y contar la frecuencia por grupos (mamíferos, aves, reptiles, etc.)

In [156]:
# Definir grupos de animales
groups = {
    'Mammals': ['cat', 'dog', 'lion', 'tiger', 'bear', 'elephant', 'giraffe', 'zebra', 
                'kangaroo', 'koala', 'panda', 'monkey', 'dolphin', 'whale', 'wolf', 
                'fox', 'rabbit', 'squirrel'],
    'Birds': ['bird', 'penguin', 'ostrich', 'owl', 'hawk', 'eagle'],
    'Reptiles': ['crocodile', 'alligator'],
    'Others': ['fish', 'shark']
}

# Contar la frecuencia por grupos
group_counts = {group: df['Animal'].isin(members).sum() for group, members in groups.items()}
group_counts_df = pd.DataFrame(list(group_counts.items()), columns=['Group', 'Count'])
print(group_counts_df)


      Group  Count
0   Mammals     40
1     Birds      8
2  Reptiles      2
3    Others      4


## 6. Identificar la posición de las primeras y últimas ocurrencias de cada animal

In [157]:
# Identificar primeras y últimas ocurrencias
first_occurrences = df['Animal'].drop_duplicates().index
last_occurrences = df.iloc[::-1]['Animal'].drop_duplicates().index[::-1]

positions_df = pd.DataFrame({
    'Animal': df['Animal'].unique(),
    'First Occurrence': first_occurrences,
    'Last Occurrence': last_occurrences
})
print(positions_df)


          Animal  First Occurrence  Last Occurrence
0            cat                 0               11
1            dog                 1               14
2           bird                 2               15
3           fish                 3               16
4           lion                 5               17
5          tiger                 6               18
6           bear                 7               19
7       elephant                20               31
8        giraffe                21               34
9          zebra                22               35
10      kangaroo                23               36
11         koala                25               37
12         panda                26               38
13        monkey                27               39
14         shark                40               40
15         whale                41               41
16       dolphin                42               42
17       penguin                43               43
18       ost

## 7. Generar un resumen estadístico de las frecuencias de los animales (media, mediana, desviación estándar)

In [158]:
# Resumen estadístico
summary_stats = frequency_counts.describe()
summary_stats['median'] = frequency_counts.median()
summary_stats['std'] = frequency_counts.std()
print(summary_stats)


count     34.000000
mean       1.764706
std        0.955330
min        1.000000
25%        1.000000
50%        1.000000
75%        3.000000
max        3.000000
median     1.000000
Name: count, dtype: float64


## 8. Filtrar y mostrar solo los animales con una frecuencia mayor que 2

In [159]:
# Animales con frecuencia mayor que 2
animals_gt_2 = frequency_counts[frequency_counts > 2]
print(animals_gt_2)


Animal
cat         3
elephant    3
dog         3
panda       3
kangaroo    3
zebra       3
giraffe     3
koala       3
tiger       3
lion        3
fish        3
bird        3
Name: count, dtype: int64


## 9. Encontrar y mostrar las etiquetas de animales que aparecen solo una vez

In [160]:
# Animales que aparecen solo una vez
animals_eq_1 = frequency_counts[frequency_counts == 1]
print(animals_eq_1)


Animal
leopard         1
hawk            1
owl             1
bat             1
squirrel        1
rabbit          1
fox             1
wolf            1
jaguar          1
penguin         1
cheetah         1
rhinoceros      1
hippopotamus    1
alligator       1
crocodile       1
ostrich         1
dolphin         1
whale           1
shark           1
eagle           1
Name: count, dtype: int64


## 10. Calcular el porcentaje de apariciones de cada animal respecto al total

In [161]:
# Porcentaje de apariciones de cada animal
total_count = df['Animal'].count()
percentage_counts = (frequency_counts / total_count) * 100
percentage_counts_df = percentage_counts.reset_index()
percentage_counts_df.columns = ['Animal', 'Percentage']
print(percentage_counts_df)


          Animal  Percentage
0            cat    5.000000
1       elephant    5.000000
2            dog    5.000000
3          panda    5.000000
4       kangaroo    5.000000
5          zebra    5.000000
6        giraffe    5.000000
7          koala    5.000000
8          tiger    5.000000
9           lion    5.000000
10          fish    5.000000
11          bird    5.000000
12          bear    3.333333
13        monkey    3.333333
14       leopard    1.666667
15          hawk    1.666667
16           owl    1.666667
17           bat    1.666667
18      squirrel    1.666667
19        rabbit    1.666667
20           fox    1.666667
21          wolf    1.666667
22        jaguar    1.666667
23       penguin    1.666667
24       cheetah    1.666667
25    rhinoceros    1.666667
26  hippopotamus    1.666667
27     alligator    1.666667
28     crocodile    1.666667
29       ostrich    1.666667
30       dolphin    1.666667
31         whale    1.666667
32         shark    1.666667
33         eag

## 11. Ordenar las etiquetas de animales por frecuencia en orden descendente

In [162]:
# Ordenar por frecuencia en orden descendente
sorted_frequency_counts = frequency_counts.sort_values(ascending=False)
print(sorted_frequency_counts)


Animal
cat             3
koala           3
elephant        3
fish            3
lion            3
tiger           3
bird            3
giraffe         3
zebra           3
kangaroo        3
panda           3
dog             3
bear            2
monkey          2
cheetah         1
shark           1
whale           1
dolphin         1
ostrich         1
crocodile       1
alligator       1
hippopotamus    1
rhinoceros      1
bat             1
penguin         1
jaguar          1
wolf            1
fox             1
rabbit          1
squirrel        1
owl             1
hawk            1
leopard         1
eagle           1
Name: count, dtype: int64


## 12. Generar un DataFrame que muestre la frecuencia acumulativa de las etiquetas

In [163]:
# Frecuencia acumulativa
cumulative_frequency = frequency_counts.cumsum()
cumulative_frequency_df = cumulative_frequency.reset_index()
cumulative_frequency_df.columns = ['Animal', 'Cumulative Count']
print(cumulative_frequency_df)


          Animal  Cumulative Count
0            cat                 3
1       elephant                 6
2            dog                 9
3          panda                12
4       kangaroo                15
5          zebra                18
6        giraffe                21
7          koala                24
8          tiger                27
9           lion                30
10          fish                33
11          bird                36
12          bear                38
13        monkey                40
14       leopard                41
15          hawk                42
16           owl                43
17           bat                44
18      squirrel                45
19        rabbit                46
20           fox                47
21          wolf                48
22        jaguar                49
23       penguin                50
24       cheetah                51
25    rhinoceros                52
26  hippopotamus                53
27     alligator    